In [ ]:
#pip install unidecode

In [16]:
import pandas as pd
import ast
from unidecode import unidecode

# YELP

In [6]:
RestaurantesY=pd.read_csv('../PROYECTO_EQUIPO/Restaurantes_Yelp.csv')

#### Generar Tabla Datos_Geográficos

In [7]:
Datos_Geograficos=RestaurantesY[['business_id','address','city','state','postal_code','latitude','longitude']].copy()

In [8]:
Datos_Geograficos.head(2)

,business_id,address,city,state,postal_code,latitude,longitude
0,MTSW4McQd7CbVtyjqoe9mw,935 Race St,Philadelphia,CA,19107,39.955505,-75.155564
1,9OG5YkX1g2GReZM0AskizA,5505 S Virginia St,Reno,AB,89502,39.476117,-119.789339


In [7]:
Datos_Geograficos.to_csv('Yelp_Datos_Geograficos.csv')

### Category

In [9]:
CategoriasYelp=RestaurantesY[['business_id','categories']].copy()

In [10]:
#obtengo todas las categorias de la columna categorias

lista_categorias = []
categorias = []
lista_categorias_limpia = []

for i in range(0, CategoriasYelp.shape[0]):
    string=CategoriasYelp["categories"].iloc[i]
    lista_categorias.append(string)

for i in range(0,len(lista_categorias)):
    #me quedo con los valores limpios de cada row para luego poner 0 o 1 segun corresponda
    lista_categorias_limpia.append(lista_categorias[i].split(','))

    #hago esto para aplicarle strip y lower
    for lista in lista_categorias_limpia:
        for string in lista:
            string=string.strip().lower()
            
    #ahora saco cada string para ir obteniendo cada categoria como string
    for string in lista_categorias[i].split(','):
        categorias.append(string.lower().strip())
        
#me quedo con un valor unico de cada categoria para que no se repita
categorias_unicas = list(set(categorias))
len(categorias_unicas) #406 categorias unicas en total

#armo un dataframe nuevo categorias para almacenar los business_id
df_categorias = pd.DataFrame()
df_categorias.insert(0,"id_restaurante" ,CategoriasYelp["business_id"])

#agrego a cada business id su correspondiente lista de categorias
df_categorias.insert(1,"categorias" ,lista_categorias_limpia)

#esto lo hice con chatgpt: transforma en lower() y aplica strip() a cada string
df_categorias["categorias"] = df_categorias["categorias"].apply(lambda x: [s.strip().lower() for s in x])

#armo df para almcacenar cada categoria como una columna con valor 0
df = pd.DataFrame(0, index=df_categorias.index, columns=categorias_unicas)

#concateno al df categorias
df_categorias = pd.concat([df_categorias, df], axis=1)

#pongo un 1 en la columna correspondiente para ir clasificando cada business id
for idx, row in df_categorias.iterrows():
    for categoria in row["categorias"]:
        if categoria in categorias_unicas:
            df_categorias.at[idx, categoria] = 1


#elimino esta columna porque ya no sirve
df_categorias.drop(columns=["categorias"], inplace=True)

In [12]:
df_categorias.sample(2)

,id_restaurante,poutineries,party & event planning,performing arts,haitian,car wash,teppanyaki,bakeries,falafel,sporting goods,...,salvadoran,home services,pet stores,pasta shops,nightlife,fondue,massage,propane,pancakes,social clubs
7272,i0b_xiKIbExKtF3vqo1Bzw,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6875,E0bcgGtrqne4MS7wt99SVA,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
#Hacer tabla

In [13]:
# Obtiene categorías principales
suma_por_categoria = df_categorias.sum()
suma_por_categoria.drop('id_restaurante', inplace=True)
suma_por_categoria.sort_values(ascending= False).head(100)

restaurants        15305
food                2986
pizza               2255
sandwiches          1940
fast food           1917
                   ...  
creperies             40
waffles               40
seafood markets       37
active life           37
filipino              37
Length: 100, dtype: object

### Cualidades (attributes) (PONERLE ATENCIÓN A ESTO)

In [14]:
df=RestaurantesY[['business_id','attributes']].copy()

In [17]:
df['attributes'] = df['attributes'].fillna('{}').apply(ast.literal_eval) #Completamos los valores faltantes en esta columna con '{}'


def convert_to_dict(val):
    # Transformamos en diccionarios reales los strings que parecen diccionarios
    if isinstance(val, str) and val.startswith("{") and val.endswith("}"):
        return ast.literal_eval(val)
    return val

df['attributes'] = df['attributes'].apply(convert_to_dict)

In [18]:
df_expanded = pd.json_normalize(df['attributes'].tolist()) #Combierte la columna en una lista de diccionarios y json_normalize pasa los valores a columnas
result_df = pd.concat([df.drop('attributes', axis=1), df_expanded], axis=1) #Combinamos el df original con el nuevo que tiene las columnas separadas

In [48]:
result_df

,business_id,RestaurantsDelivery,OutdoorSeating,BusinessAcceptsCreditCards,BusinessParking,BikeParking,RestaurantsPriceRange2,RestaurantsTakeOut,ByAppointmentOnly,WiFi,...,CoatCheck,BestNights,Smoking,BYOBCorkage,Corkage,Open24Hours,DietaryRestrictions,AgesAllowed,RestaurantsCounterService,AcceptsInsurance
0,MTSW4McQd7CbVtyjqoe9mw,False,False,False,"{'garage': False, 'street': True, 'validated':...",True,1,True,False,u'free',...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9OG5YkX1g2GReZM0AskizA,True,True,True,"{'garage': False, 'street': False, 'validated'...",True,2,True,NaN,u'free',...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,tMkwHmWFUEXrC9ZduonpTg,None,False,True,"{'garage': True, 'street': False, 'validated':...",False,NaN,None,NaN,u'free',...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2xVsWBNFwZOxIOdd9Mwnww,False,True,True,"{'garage': False, 'street': False, 'validated'...",NaN,2,True,NaN,u'free',...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ljxNT9p0y7YMPx0fcNBGig,False,True,True,"{'garage': False, 'street': True, 'validated':...",True,2,True,NaN,'free',...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15300,wVxXRFf10zTTAs11nr4xeA,True,False,True,"{'garage': False, 'street': True, 'validated':...",True,2,True,NaN,'no',...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15301,sf_oQ62L8UEnOOLf00nNGA,True,False,True,"{'garage': False, 'street': False, 'validated'...",NaN,1,True,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15302,l9eLGG9ZKpLJzboZq-9LRQ,True,NaN,True,"{'garage': False, 'street': False, 'validated'...",True,1,True,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15303,WnT9NIzQgLlILjPT0kEcsQ,True,False,True,"{'garage': None, 'street': True, 'validated': ...",True,2,True,NaN,u'free',...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
result_df.columns

Index(['business_id', 'RestaurantsDelivery', 'OutdoorSeating',
       'BusinessAcceptsCreditCards', 'BusinessParking', 'BikeParking',
       'RestaurantsPriceRange2', 'RestaurantsTakeOut', 'ByAppointmentOnly',
       'WiFi', 'Alcohol', 'Caters', 'RestaurantsGoodForGroups',
       'RestaurantsAttire', 'NoiseLevel', 'GoodForKids',
       'RestaurantsReservations', 'GoodForMeal', 'RestaurantsTableService',
       'HappyHour', 'DogsAllowed', 'Ambience', 'HasTV',
       'BusinessAcceptsBitcoin', 'WheelchairAccessible', 'BYOB', 'DriveThru',
       'GoodForDancing', 'Music', 'CoatCheck', 'BestNights', 'Smoking',
       'BYOBCorkage', 'Corkage', 'Open24Hours', 'DietaryRestrictions',
       'AgesAllowed', 'RestaurantsCounterService', 'AcceptsInsurance'],
      dtype='object')

Obtener datos parecidos:
['Service options', 'Health & safety', 'Highlights', 'Popular for', 'Offerings', 'Amenities', 'Atmosphere', 'Crowd','Dining options', 'Payments', 'Planning']

YELP:
RestaurantsDelivery,RestaurantsTakeOut,BusinessAcceptsCreditCards,ByAppointmentOnly,RestaurantsReservations,RestaurantsGoodForGroups

Eliminar: RestaurantsCounterService, AcceptsInsurance,

In [35]:
result_df['AgesAllowed'].isna().sum()

15290

In [ ]:
result_df[-result_df['AgesAllowed'].isna()]

In [47]:
result_df['AgesAllowed'].loc[14333]

"u'allages'"

In [25]:
result_df.shape

(15305, 39)

In [21]:
result_df['RestaurantsDelivery'].unique()

array(['False', 'True', 'None', nan], dtype=object)

In [38]:
result_df.shape

(15305, 39)

In [37]:
result_df['NoiseLevel'][5948]

nan

In [30]:
result_df['NoiseLevel'].unique()

array([nan, "u'average'", "u'quiet'", "'average'", "u'very_loud'",
       "'quiet'", 'None', "u'loud'", "'loud'", "'very_loud'"],
      dtype=object)

In [50]:
import numpy as np

In [1]:
def dicc(row):
    #result = json.loads(row)
    result = ast.literal_eval(row)
    return result

In [21]:
#CÓDIFO DE REPOSITORIO MAICO
def etl_atributtes(business):
    atributtes = pd.json_normalize(data = business['attributes'])
    atributtes['BusinessParking'].fillna("{'garage': False, 'street': False, 'validated': False, 'lot': False, 'valet': False}", inplace=True)
    atributtes.loc[atributtes['BusinessParking'] == 'None', 'BusinessParking'] = "{'garage': False, 'street': False, 'validated': False, 'lot': False, 'valet': False}"

    atributtes.loc[atributtes['Ambience'] == 'None', 'Ambience'] = "{'romantic': False, 'intimate': False, 'classy': False, 'hipster': False, 'touristy': False, 'trendy': False, 'upscale': False, 'casual': False}"
    atributtes['Ambience'] = atributtes['Ambience'].fillna("{'romantic': False, 'intimate': False, 'classy': False, 'hipster': False, 'touristy': False, 'trendy': False, 'upscale': False, 'casual': False}")

    garage = pd.json_normalize(atributtes.BusinessParking.apply(dicc))
    ambience = pd.json_normalize(atributtes['Ambience'].apply(dicc))

    atributtes['garage'] = garage['garage']
    atributtes['garage'].fillna(0, inplace=True)

    atributtes.drop(['BusinessParking', 'Ambience'], axis=1, inplace=True)

    ambience['good_ambience'] = ambience['romantic'] + ambience['intimate'] + ambience['classy'] + ambience['hipster'] + ambience['touristy'] + ambience['trendy'] + ambience['upscale'] + ambience['casual'] + ambience['divey']

    ambience.loc[ambience['good_ambience'] > 1, 'good_ambience'] = 1
    ambience.fillna(0, inplace=True)
    atributtes['good_ambience'] = ambience['good_ambience']

    atributtes['RestaurantsTakeOut'].fillna(0, inplace=True)
    atributtes['RestaurantsDelivery'].fillna(0, inplace=True)
    atributtes['RestaurantsTakeOut'] = atributtes['RestaurantsTakeOut'].map({'True': 1, 'False': 0, 'None': 0})
    atributtes['RestaurantsDelivery'] = atributtes['RestaurantsDelivery'].map({'True': 1, 'False': 0, 'None': 0})
    atributtes['delivery'] = atributtes['RestaurantsTakeOut'] + atributtes['RestaurantsDelivery']
    atributtes['delivery'] = pd.to_numeric(atributtes['delivery'], errors='coerce').fillna(0).astype(int)
    atributtes['delivery'] = atributtes['delivery'].replace(to_replace=2, value=1)


    atributtes.drop(['RestaurantsTakeOut', 'RestaurantsDelivery'], axis=1, inplace=True)

    top20 = atributtes.notna().sum().sort_values(ascending=False).head(20).index.tolist()

    atributtes = atributtes[top20]

    atributtes.fillna(0, inplace=True)
    atributtes.replace('None', 0, inplace=True)
    atributtes.replace('False', 0, inplace=True)
    atributtes.replace(False, 0, inplace=True)
    atributtes.replace('True', 1, inplace=True)

    atributtes['WiFi'] = np.where(atributtes['WiFi'] != 0, 1, 0)

    atributtes.loc[atributtes['Alcohol'] == "u'none'", 'Alcohol'] = 0
    atributtes.loc[atributtes['Alcohol'] == "'none'", 'Alcohol'] = 0
    atributtes.loc[atributtes['Alcohol'] != 0, 'Alcohol'] = 1
    atributtes['Alcohol'].unique()

    atributtes.loc[atributtes['NoiseLevel'].str.contains('quiet').fillna(False), 'NoiseLevel'] = 1
    atributtes.loc[atributtes['NoiseLevel'].str.contains('average').fillna(False), 'NoiseLevel'] = 2
    atributtes.loc[atributtes['NoiseLevel'].str.contains('loud').fillna(False), 'NoiseLevel'] = 3
    atributtes.loc[atributtes['RestaurantsAttire'] != 0,'RestaurantsAttire'] = 1
    atributtes.loc[atributtes['garage'] != 0,'garage'] = 1
    atributtes.loc[atributtes['GoodForMeal'] == 0,'GoodForMeal'] = "{'dessert': False, 'latenight': False, 'lunch': False, 'dinner': False, 'brunch': False, 'breakfast': False}"
    atributtes['GoodForMeal'].fillna("{'dessert': False, 'latenight': False, 'lunch': False, 'dinner': False, 'brunch': False, 'breakfast': False}", inplace=True)
    meal_types = pd.json_normalize(atributtes['GoodForMeal'].apply(dicc))
    atributtes.drop(['GoodForMeal'], axis=1, inplace=True)
    atributtes['meal_diversity'] = meal_types['dessert'] + meal_types['latenight'] + meal_types['lunch'] + meal_types['dinner'] + meal_types['brunch'] + meal_types['breakfast']
    atributtes['meal_diversity'].fillna(0, inplace=True)
    atributtes['business_id'] = business['business_id']
    return atributtes

In [49]:
etl_atributtes(df)

,delivery,good_ambience,garage,BusinessAcceptsCreditCards,RestaurantsPriceRange2,OutdoorSeating,RestaurantsReservations,RestaurantsGoodForGroups,GoodForKids,HasTV,...,Alcohol,WiFi,NoiseLevel,BikeParking,Caters,RestaurantsTableService,WheelchairAccessible,HappyHour,meal_diversity,business_id
0,1,0,0,0,1,0,0,0,0,0,...,0,1,0,1,1,0,0,0,0,MTSW4McQd7CbVtyjqoe9mw
1,1,0,0,1,2,1,1,1,1,1,...,1,1,2,1,1,1,0,1,0,9OG5YkX1g2GReZM0AskizA
2,0,1,1,1,0,0,1,1,0,0,...,1,1,2,0,1,1,1,1,0,tMkwHmWFUEXrC9ZduonpTg
3,1,0,0,1,2,1,0,1,1,1,...,1,1,2,0,0,0,0,1,0,2xVsWBNFwZOxIOdd9Mwnww
4,1,1,0,1,2,1,1,1,1,1,...,1,1,2,1,1,1,1,1,1,ljxNT9p0y7YMPx0fcNBGig
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15300,1,0,0,1,2,0,0,0,1,1,...,0,1,2,1,1,0,0,0,2,wVxXRFf10zTTAs11nr4xeA
15301,1,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,sf_oQ62L8UEnOOLf00nNGA
15302,1,0,0,1,1,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,l9eLGG9ZKpLJzboZq-9LRQ
15303,1,0,0,1,2,0,0,1,1,1,...,0,1,0,1,1,1,0,0,0,WnT9NIzQgLlILjPT0kEcsQ


In [129]:
result_df.shape

(15305, 39)

Alcohol', 'Caters', 'RestaurantsGoodForGroups',
       'RestaurantsAttire', 'NoiseLevel', 'GoodForKids',
       'RestaurantsReservations', 'GoodForMeal', 'RestaurantsTableService',
       'HappyHour', 'DogsAllowed', 'Ambience', 'HasTV',
       'BusinessAcceptsBitcoin', 'WheelchairAccessible', 'BYOB', 'DriveThru',
       'GoodForDancing', 'Music',

In [144]:
#Revisar WiFi	BusinessParking
result_df['Alcohol'].unique()

array(["u'none'", "'full_bar'", "u'full_bar'", "'beer_and_wine'", nan,
       "u'beer_and_wine'", "'none'", 'None'], dtype=object)

In [149]:
result_df['WiFi'].unique()

array(["u'free'", "'free'", "'no'", nan, "u'no'", 'None', "'paid'",
       "u'paid'"], dtype=object)

In [148]:
result_df['RestaurantsGoodForGroups'].unique()

array([nan, 'True', 'False', 'None'], dtype=object)

In [116]:
result_df.columns

Index(['RestaurantsDelivery', 'OutdoorSeating', 'BusinessAcceptsCreditCards',
       'BusinessParking', 'BikeParking', 'RestaurantsPriceRange2',
       'RestaurantsTakeOut', 'ByAppointmentOnly', 'WiFi', 'Alcohol', 'Caters',
       'RestaurantsGoodForGroups', 'RestaurantsAttire', 'NoiseLevel',
       'GoodForKids', 'RestaurantsReservations', 'GoodForMeal',
       'RestaurantsTableService', 'HappyHour', 'DogsAllowed', 'Ambience',
       'HasTV', 'BusinessAcceptsBitcoin', 'WheelchairAccessible', 'BYOB',
       'DriveThru', 'GoodForDancing', 'Music', 'CoatCheck', 'BestNights',
       'Smoking', 'BYOBCorkage', 'Corkage', 'Open24Hours',
       'DietaryRestrictions', 'AgesAllowed', 'RestaurantsCounterService',
       'AcceptsInsurance'],
      dtype='object')

In [117]:
result_df['GoodForMeal'][4]

"{'dessert': False, 'latenight': False, 'lunch': False, 'dinner': True, 'brunch': False, 'breakfast': False}"

### Horarios

In [232]:
Horarios=RestaurantesY[['business_id','hours']].copy()

In [227]:
Horarios['hours'].isna().sum()

2743

In [229]:
Horarios['hours'].fillna("{'Monday': 'Sin info', 'Tuesday': 'Sin info', 'Wednesday': 'Sin info', 'Thursday': 'Sin info', 'Friday': 'Sin info', 'Saturday': 'Sin info', 'Sunday': 'Sin info'}", inplace=True)

In [204]:
import numpy as np

In [230]:
dataframes_individuales = []

for _, row in Horarios.iterrows():
    try:
        lista_horarios = ast.literal_eval(row['hours'])
        df = pd.DataFrame(list(lista_horarios.items()), columns=['Dia', 'Horario'])
        df['business_id'] = row['business_id']
        #df.replace('NaN', np.nan, inplace=True)
        dataframes_individuales.append(df)
    except (ValueError, SyntaxError):
        pass  # Ignora las filas con valores NaN o cadenas mal formadas.

# Concatena todos los DataFrames individuales en uno solo.
tabla_pivot_total = pd.concat(dataframes_individuales)

# Pivotea la tabla.
Tabla_Horarios = tabla_pivot_total.pivot(index='business_id', columns='Dia', values='Horario')

In [241]:
Tabla_Horarios.replace(np.nan, 'Sin info', inplace=True)

In [253]:
Tabla_Horarios.sample(2)

Dia,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
business_id,,,,,,,
w4nHq_elJwPpI4iZ6ro3JQ,8:0-18:0,Cerrado,8:0-18:0,9:0-16:0,8:0-18:0,Sin info,8:0-18:0
7aDtfwkv9mZnb4wtlbjBMQ,17:0-22:0,17:0-21:0,17:0-22:0,Sin info,17:0-21:0,17:0-21:0,17:0-21:0


In [247]:
# Se asume que cuando hay un '0:0-0:0' es porque pueda estar cerrado.
Tabla_Horarios.replace('0:0-0:0', 'Cerrado', inplace=True)

In [254]:
Tabla_Horarios.to_csv('Yelp_Horarios.csv')

### Restaurantes

Borrar probablemente

In [256]:
Restaurantes_Yelp=RestaurantesY[['business_id','name']].copy()

In [260]:
Restaurantes_Yelp.sample(1)

,business_id,name
5029,Nl8lPfJalOiX9rJiibD8jw,Westampton Family Diner


### Otras Tablas

In [341]:
A=pd.read_csv('../PROYECTO_EQUIPO/Restaurantes_Yelp_Reviews.csv')

In [342]:
A.sample(2)

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
738667,OgKTQE_qCCMGsuB-oqp3Kg,_t2pN38M6ATZCLtRnfalRA,2Ugp3An60FEIQM2FN92AsA,5.0,0,0,0,What a surprise! Was starving and happened upo...,2017-01-31
779145,IMnC8CxA6MyzYr9ikSXT2w,2Ttdml12syjb782j9lIxuw,rH8dKfQZOdpL_i9VaNWQdw,5.0,1,0,0,I love that I can mix n match sliders....the c...,2016-05-19


In [343]:
B=pd.read_csv('../PROYECTO_EQUIPO/Restaurantes_Yelp_Tips.csv')

In [344]:
B.sample(2)

,user_id,business_id,text,date,compliment_count
181667,7Ju_RQfW5vEo1ExxauUQMg,eT1nr9nFklcUg8hefYcQJA,Sage tea is not to be missed. Everything on t...,2012-05-16,0
388158,W-hT988aIrWxmYxbR6CT4Q,NDIiNcJZpdoQpXGXl7IA2Q,They have an amazing jalapeño margarita. The f...,2015-05-06,0


In [345]:
C=pd.read_csv('../PROYECTO_EQUIPO/Restaurantes_User_Yelp.csv')

In [348]:
C.sample(2)

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
83777,6bYg1vNuBTXlwXA-dDnLkg,Sammie,21,2014-07-09,11,0,0,NaN,YeFtMRfNCWuOMHMeybqPXw,1,...,0,0,0,0,0,0,0,0,0,0
313414,gIPJlEtqqvQKmi8pwT-mzw,Laura,16,2013-03-18,8,1,2,NaN,HRMHEVgdt3-xEZi8ewLQfA,0,...,0,0,0,0,0,0,0,0,0,0


### REVISAR CUANTOS RESTAURANTES DE GOOGLE ENCONTRAMOS EN YELP

### Juntar los restaurantes 

In [377]:
Restaurantes_G=pd.read_csv('../PROYECTO_EQUIPO/Restaurantes_Google.csv')

In [378]:
Restaurantes_Google=Restaurantes_G[['gmap_id','name','latitude','longitude']].copy()

In [379]:
Restaurantes_G.shape

(3223, 11)

In [380]:
RestaurantesYelpC=RestaurantesY[['business_id','name','latitude','longitude']].copy()

In [381]:
# Supongamos que tienes los DataFrames Restaurantes_Google y RestaurantesYelpC

# Normalización y limpieza de nombres
Restaurantes_Google['name'] = Restaurantes_Google['name'].str.lower().apply(unidecode)
RestaurantesYelpC['name'] = RestaurantesYelpC['name'].str.lower().apply(unidecode)

In [382]:
Restaurantes_Google.sample(1)

,gmap_id,name,latitude,longitude
375,0x8803aa0a9fb8fbfb:0xe71990029b83660c,subway,44.326872,-88.168373


In [383]:
RestaurantesYelpC.sample(1)

,business_id,name,latitude,longitude
10029,E2iT9ujoMDPm4NgcIBjTtQ,little caesar's pizza,38.69223,-90.393356


In [389]:
# Merge para obtener los restaurantes comunes por 'name_normalized', 'latitude' y 'longitude'
restaurantes_comunes = pd.merge(Restaurantes_Google, RestaurantesYelpC, on=['name','latitude','longitude'], how='inner')

In [390]:
restaurantes_comunes

,gmap_id,name,latitude,longitude,business_id
0,0x80993f359f5361a3:0xf4b5055a79239dbf,399 sushi,39.466836,-119.784719,qQpgF5fGMTulUUtjzlsnwg
